# 07_EWSINDy_STLSQ - Physics-SR Framework v4.1

## Stage 2.4: E-WSINDy Sparse Selection on Augmented Library

**Author:** Zhengze Zhang  
**Affiliation:** Department of Statistics, Columbia University  
**Contact:** zz3239@columbia.edu  
**Date:** January 2026  
**Version:** 4.1 (Three-Layer Sparse Selection Enhancement)

---

### Purpose

Noise-robust equation discovery via weak-form sparse regression on augmented library.
This module provides a **three-layer enhancement** for robust sparse selection.

### v4.1 Three-Layer Enhancement

| Layer | Component | Purpose |
|-------|-----------|----------|
| **Layer 1** | Relative Thresholding | Scale-invariant STLSQ thresholding |
| **Layer 2** | E-SINDy Ensemble | Bootstrap-based robust selection with inclusion probabilities |
| **Layer 3** | EBIC/CV Selection | Automatic threshold selection via information criteria |

### v4.1 Key Classes and Functions

| Component | Type | Description |
|-----------|------|-------------|
| `EWSINDySTLSQ` | Class | STLSQ with relative threshold (Layer 1) |
| `ESINDyEnsemble` | Class | Bootstrap ensemble selection (Layer 2) |
| `select_threshold_by_ebic()` | Function | EBIC-based threshold selection (Layer 3) |
| `select_threshold_by_cv()` | Function | CV-based threshold selection (Layer 3) |

### Weak Form Theory

**Strong form** (noise-sensitive):
$$\frac{\partial q}{\partial t} = f(q, \nabla q, \nabla^2 q)$$

**Weak form** (noise-robust): Multiply by test function $\psi$ and integrate by parts:
$$\int \psi \cdot \nabla^2 q \, dx = -\int \nabla\psi \cdot \nabla q \, dx + \text{boundary terms}$$

**Result:** Derivatives transferred from noisy data $q$ to smooth test function $\psi$.

### Key Properties of v4.1 Design

1. **Relative Threshold:** `threshold = base_threshold * max(|coef|)` makes STLSQ robust to coefficient scale
2. **Ensemble Selection:** Bootstrap aggregation provides inclusion probabilities for natural UQ
3. **EBIC Selection:** Automatic threshold tuning balances fit vs. complexity
4. **Can KEEP correct PySR terms:** If PySR found sin(x^2) and it's correct, E-WSINDy will select it
5. **Can DISCOVER missed terms:** If PySR missed x*z, E-WSINDy can find it in polynomial layer
6. **Can REJECT errors:** If PySR included spurious term, E-WSINDy's sparsity can exclude it

### References

- Messenger, D. A., & Bortz, D. M. (2021). Weak SINDy for PDEs. *JCP*, 443, 110525.
- Fasel et al. (2022). Ensemble-SINDy. *Proc. R. Soc. A.*
- Chen, J., & Chen, Z. (2008). Extended BIC for large model spaces. *Biometrika*, 95(3).
- Framework v4.0/v4.1 Section 4.4: E-WSINDy Sparse Selection

---
## Section 1: Header and Imports

In [ ]:
"""
07_EWSINDy_STLSQ.ipynb - E-WSINDy with STLSQ on Augmented Library
===================================================================

Three-Stage Physics-Informed Symbolic Regression Framework v4.1

This module provides:
- EWSINDySTLSQ: Weak-form SINDy with STLSQ sparse regression
- Source attribution via analyze_selection_sources()
- 50-1000x noise robustness improvement over finite differences
- Exact sparsity (true zeros) via iterative thresholding

v4.1 Key Changes from v3.0:
- Now accepts library_names with source tags [PySR], [Var], [Poly], [Op]
- New method: analyze_selection_sources()
- Returns selection_analysis in output dictionary
- New parameter: normalize_columns (default: True)

Output Format:
- coefficients, support, equation, r_squared (same as v3.0)
- selection_analysis: Dict with from_pysr, from_variant, from_poly, from_op counts

Author: Zhengze Zhang
Affiliation: Department of Statistics, Columbia University
Contact: zz3239@columbia.edu
"""

# Import core module
%run 00_Core.ipynb

In [ ]:
# Additional imports for E-WSINDy
from scipy import integrate
from scipy.interpolate import interp1d
from sklearn.linear_model import Lasso, Ridge
from typing import Dict, List, Tuple, Optional, Any

print("07_EWSINDy_STLSQ v4.1: Additional imports successful.")

---
## Section 2: Class Definition

In [ ]:
# ==============================================================================
# E-WSINDY STLSQ CLASS (v4.2 HYBRID)
# ==============================================================================
# 
# v4.2 HYBRID: Adaptive term limiting using percentile threshold
#              with MIN_TERMS and MAX_TERMS bounds
#
# Strategy:
#   1. Compute 80th percentile of |coefficients|
#   2. Keep terms above this percentile
#   3. Ensure result is within [MIN_TERMS, MAX_TERMS] bounds
#
# Author: Zhengze Zhang
# Contact: zz3239@columbia.edu
# ==============================================================================

class EWSINDySTLSQ:
    """
    E-WSINDy with STLSQ: Weak-form Sparse Regression (v4.2 Hybrid).
    
    Now operates on augmented library from v4.0 and includes
    source attribution for selected terms.
    
    v4.2 HYBRID: Adaptive term limiting using percentile threshold
    to prevent overfitting while maintaining flexibility.
    
    Provides 50-1000x noise improvement over strong-form methods.
    
    Attributes
    ----------
    threshold : float
        STLSQ sparsity threshold (default: 0.1)
    max_iter : int
        Maximum STLSQ iterations (default: 20)
    n_test_functions : int
        Number of test functions for weak form (default: 50)
    test_function_type : str
        Type of test function: 'gaussian' or 'polynomial'
    test_function_width : float
        Width parameter for test functions (default: 0.1)
    use_weak_form : bool
        Whether to use weak form transformation (default: True)
    normalize_columns : bool
        Whether to normalize feature columns (default: True, v4.1)
    
    Methods
    -------
    fit(feature_library, y, library_names, t) -> Dict
        Fit E-WSINDy model using STLSQ
    analyze_selection_sources(support, library_names) -> Dict
        Analyze where selected terms originated (v4.1)
    get_equation() -> str
        Get string representation of equation
    predict(Phi_new) -> np.ndarray
        Make predictions
    
    Reference
    ---------
    Messenger & Bortz (2021). Multiscale Modeling & Simulation.
    Framework v4.0/v4.1 Section 4.4: E-WSINDy Sparse Selection
    
    Examples
    --------
    >>> model = EWSINDySTLSQ(threshold=0.1)
    >>> result = model.fit(Phi_aug, y, library_names=names)
    >>> print(f"Selection analysis: {result['selection_analysis']}")
    """
    
    # v4.2 HYBRID: Adaptive term limiting parameters
    MIN_TERMS = 5
    MAX_TERMS = 20
    COEF_PERCENTILE = 80
    
    def __init__(
        self,
        threshold: float = DEFAULT_STLSQ_THRESHOLD,
        max_iter: int = DEFAULT_STLSQ_MAX_ITER,
        n_test_functions: int = 50,
        test_function_type: str = 'gaussian',
        test_function_width: float = 0.1,
        use_weak_form: bool = True,
        normalize_columns: bool = True,
        relative_threshold: bool = True,
        ridge_alpha: float = 1e-6
    ):
        """
        Initialize EWSINDySTLSQ.
        
        Parameters
        ----------
        threshold : float
            Base sparsity threshold. If relative_threshold=True, this is
            multiplied by max(|coefficients|). Default: 0.1
        max_iter : int
            Maximum number of STLSQ iterations.
            Default: 20
        n_test_functions : int
            Number of test functions for weak form.
            Default: 50
        test_function_type : str
            'gaussian' for Gaussian bumps, 'polynomial' for polynomial.
            Default: 'gaussian'
        test_function_width : float
            Width of Gaussian bumps (as fraction of domain).
            Default: 0.1
        use_weak_form : bool
            Whether to use weak form (True) or standard form (False).
            Default: True
        normalize_columns : bool
            Whether to normalize feature library columns (v4.1).
            Default: True
        relative_threshold : bool
            If True, threshold = threshold * max(|coef|). This makes
            STLSQ robust to coefficient scale variations (v4.1 FIX).
            Default: True
        ridge_alpha : float
            Ridge regularization for initial OLS (stability).
            Default: 1e-6
        """
        self.threshold = threshold
        self.max_iter = max_iter
        self.n_test_functions = n_test_functions
        self.test_function_type = test_function_type
        self.test_function_width = test_function_width
        self.use_weak_form = use_weak_form
        self.normalize_columns = normalize_columns
        self.relative_threshold = relative_threshold
        self.ridge_alpha = ridge_alpha
        
        # Internal state
        self._coefficients = None
        self._support = None
        self._library_names = None
        self._n_features = None
        self._n_iterations = 0
        self._convergence_history = []
        self._fit_complete = False
        self._r2_score = None
        self._mse = None
        self._selection_analysis = None
        self._column_scales = None
        self._effective_threshold = None
        self._terms_limited = False  # v4.2: Track if limiting was applied
    
    def fit(
        self,
        feature_library: np.ndarray,
        y: np.ndarray,
        t: np.ndarray = None,
        library_names: List[str] = None
    ) -> Dict[str, Any]:
        """
        Fit E-WSINDy model using STLSQ.
        
        Parameters
        ----------
        feature_library : np.ndarray
            Feature library (augmented or standard)
        y : np.ndarray
            Target vector
        t : np.ndarray, optional
            Time vector for weak form (if None, uses indices)
        library_names : List[str], optional
            Feature names with source tags for attribution
            
        Returns
        -------
        Dict
            - coefficients: Sparse coefficient vector
            - support: Boolean mask of active terms
            - equation: Formatted equation string
            - selection_analysis: Source attribution (v4.1)
            - r_squared: Coefficient of determination
            - n_iterations: Convergence iterations
            - weak_form_Q: Weak-form feature matrix (if used)
            - weak_form_b: Weak-form target vector (if used)
        """
        n_samples, n_features = feature_library.shape
        self._n_features = n_features
        
        # Set feature names
        if library_names is None:
            self._library_names = [f'f{i}' for i in range(n_features)]
        else:
            self._library_names = list(library_names)
        
        # Generate time vector if not provided
        if t is None:
            t = self._generate_time_vector(n_samples)
        
        # Normalize columns if requested (v4.1)
        Phi_normalized = feature_library.copy()
        if self.normalize_columns:
            Phi_normalized, self._column_scales = self._normalize_library(feature_library)
        else:
            self._column_scales = np.ones(n_features)
        
        # Apply weak form transformation if enabled
        if self.use_weak_form:
            Q, b = self._weak_form_transform(Phi_normalized, y, t)
        else:
            # Standard form: direct regression
            Q = Phi_normalized
            b = y
        
        # Run STLSQ
        self._coefficients = self._stlsq_iteration(Q, b)
        
        # Rescale coefficients if normalized
        if self.normalize_columns:
            self._coefficients = self._coefficients / self._column_scales
        
        self._support = np.abs(self._coefficients) > 0
        
        # ======================================================================
        # v4.2 HYBRID: Adaptive term limiting
        # ======================================================================
        n_selected_original = np.sum(self._support)
        self._terms_limited = False
        
        if n_selected_original > self.MAX_TERMS:
            self._terms_limited = True
            coef_abs = np.abs(self._coefficients)
            nonzero_coefs = coef_abs[self._support]
            
            # Compute percentile threshold
            percentile_threshold = np.percentile(nonzero_coefs, self.COEF_PERCENTILE)
            
            # Apply percentile threshold
            candidate_support = (coef_abs >= percentile_threshold) & (coef_abs > 0)
            n_candidates = np.sum(candidate_support)
            
            # Ensure within [MIN_TERMS, MAX_TERMS] bounds
            if n_candidates < self.MIN_TERMS:
                # Keep top MIN_TERMS by coefficient magnitude
                nonzero_indices = np.where(self._support)[0]
                top_k_local = np.argsort(coef_abs[nonzero_indices])[-self.MIN_TERMS:]
                top_k_global = nonzero_indices[top_k_local]
                new_support = np.zeros(n_features, dtype=bool)
                new_support[top_k_global] = True
            elif n_candidates > self.MAX_TERMS:
                # Keep top MAX_TERMS by coefficient magnitude
                nonzero_indices = np.where(self._support)[0]
                top_k_local = np.argsort(coef_abs[nonzero_indices])[-self.MAX_TERMS:]
                top_k_global = nonzero_indices[top_k_local]
                new_support = np.zeros(n_features, dtype=bool)
                new_support[top_k_global] = True
            else:
                new_support = candidate_support
            
            # Update support
            self._support = new_support
            
            # Re-estimate coefficients with limited support
            if np.sum(self._support) > 0:
                Phi_selected = feature_library[:, self._support]
                new_coefficients = np.zeros(n_features)
                try:
                    new_coefficients[self._support] = np.linalg.lstsq(
                        Phi_selected, y, rcond=None
                    )[0]
                    self._coefficients = new_coefficients
                except:
                    pass  # Keep original if lstsq fails
        # ======================================================================
        
        # Compute metrics on original scale
        y_pred = feature_library @ self._coefficients
        self._mse = np.mean((y - y_pred)**2)
        ss_tot = np.sum((y - np.mean(y))**2)
        ss_res = np.sum((y - y_pred)**2)
        self._r2_score = 1 - ss_res / ss_tot if ss_tot > 0 else 0.0
        
        # Analyze selection sources (v4.1)
        self._selection_analysis = self.analyze_selection_sources(
            self._support, self._library_names
        )
        
        self._fit_complete = True
        
        result = {
            'coefficients': self._coefficients,
            'support': self._support,
            'equation': self.get_equation(),
            'selection_analysis': self._selection_analysis,
            'n_active_terms': int(np.sum(self._support)),
            'n_iterations': self._n_iterations,
            'r_squared': self._r2_score,
            'r2_score': self._r2_score,  # Alias for compatibility
            'mse': self._mse,
            'convergence_history': self._convergence_history,
            'threshold': self.threshold,
            'terms_limited': self._terms_limited,  # v4.2
            'n_selected_before_limit': n_selected_original  # v4.2
        }
        
        # Add weak form matrices if used
        if self.use_weak_form:
            result['weak_form_Q'] = Q
            result['weak_form_b'] = b
        
        return result
    
    def analyze_selection_sources(
        self,
        support: np.ndarray,
        library_names: List[str]
    ) -> Dict[str, int]:
        """
        Analyze where selected terms originated (v4.1).
        
        Parameters
        ----------
        support : np.ndarray
            Boolean mask of selected terms
        library_names : List[str]
            Feature names with source tags
            
        Returns
        -------
        Dict[str, int]
            - from_powerlaw: Count of [PowLaw] terms selected (v4.1 NEW)
            - from_pysr: Count of [PySR] terms selected
            - from_variant: Count of [Var] terms selected
            - from_poly: Count of [Poly] terms selected
            - from_op: Count of [Op] terms selected
            - from_unknown: Count of untagged terms selected
            - total_selected: Total selected terms
        """
        sources = {
            'from_powerlaw': 0,
            'from_pysr': 0,
            'from_variant': 0,
            'from_poly': 0,
            'from_op': 0,
            'from_unknown': 0,
            'total_selected': 0
        }
        
        selected_indices = np.where(support)[0]
        sources['total_selected'] = len(selected_indices)
        
        for idx in selected_indices:
            if idx >= len(library_names):
                sources['from_unknown'] += 1
                continue
                
            name = library_names[idx]
            if name.startswith('[PowLaw]'):
                sources['from_powerlaw'] += 1
            elif name.startswith('[PySR]'):
                sources['from_pysr'] += 1
            elif name.startswith('[Var]'):
                sources['from_variant'] += 1
            elif name.startswith('[Poly]'):
                sources['from_poly'] += 1
            elif name.startswith('[Op]'):
                sources['from_op'] += 1
            else:
                sources['from_unknown'] += 1
        
        return sources
    
    def _normalize_library(
        self,
        Phi: np.ndarray
    ) -> Tuple[np.ndarray, np.ndarray]:
        """
        Normalize feature library columns to unit variance.
        
        Parameters
        ----------
        Phi : np.ndarray
            Feature library matrix
            
        Returns
        -------
        Tuple[np.ndarray, np.ndarray]
            (normalized_Phi, column_scales)
        """
        scales = np.std(Phi, axis=0)
        scales[scales < 1e-10] = 1.0  # Avoid division by zero
        
        normalized = Phi / scales
        return normalized, scales
    
    def _generate_time_vector(
        self,
        n_samples: int
    ) -> np.ndarray:
        """
        Generate uniform time vector.
        
        Parameters
        ----------
        n_samples : int
            Number of samples
        
        Returns
        -------
        np.ndarray
            Time vector from 0 to 1
        """
        return np.linspace(0, 1, n_samples)
    
    def _generate_test_functions(
        self,
        t: np.ndarray
    ) -> Tuple[np.ndarray, np.ndarray]:
        """
        Generate test functions and their derivatives.
        
        Parameters
        ----------
        t : np.ndarray
            Time vector
        
        Returns
        -------
        Tuple[np.ndarray, np.ndarray]
            - psi: Test functions of shape (n_test, n_samples)
            - dpsi: Derivatives of shape (n_test, n_samples)
        """
        n_samples = len(t)
        t_min, t_max = t.min(), t.max()
        t_range = t_max - t_min
        
        # Centers for test functions (avoid boundaries)
        centers = np.linspace(
            t_min + 0.1 * t_range,
            t_max - 0.1 * t_range,
            self.n_test_functions
        )
        
        width = self.test_function_width * t_range
        
        psi = np.zeros((self.n_test_functions, n_samples))
        dpsi = np.zeros((self.n_test_functions, n_samples))
        
        for m, center in enumerate(centers):
            if self.test_function_type == 'gaussian':
                psi[m], dpsi[m] = self._gaussian_bump(t, center, width)
            else:
                psi[m], dpsi[m] = self._polynomial_bump(t, center, width)
        
        return psi, dpsi
    
    def _gaussian_bump(
        self,
        t: np.ndarray,
        center: float,
        width: float
    ) -> Tuple[np.ndarray, np.ndarray]:
        """
        Generate Gaussian bump test function.
        
        psi(t) = exp(-(t - center)^2 / (2 * width^2))
        dpsi(t) = -(t - center) / width^2 * psi(t)
        
        Parameters
        ----------
        t : np.ndarray
            Time vector
        center : float
            Center of Gaussian
        width : float
            Width (standard deviation)
        
        Returns
        -------
        Tuple[np.ndarray, np.ndarray]
            (psi, dpsi)
        """
        z = (t - center) / width
        psi = np.exp(-0.5 * z**2)
        dpsi = -z / width * psi
        return psi, dpsi
    
    def _polynomial_bump(
        self,
        t: np.ndarray,
        center: float,
        width: float
    ) -> Tuple[np.ndarray, np.ndarray]:
        """
        Generate polynomial bump test function.
        
        Uses (1 - ((t-center)/width)^2)^4 for compact support.
        
        Parameters
        ----------
        t : np.ndarray
            Time vector
        center : float
            Center of bump
        width : float
            Half-width of support
        
        Returns
        -------
        Tuple[np.ndarray, np.ndarray]
            (psi, dpsi)
        """
        z = (t - center) / width
        mask = np.abs(z) < 1
        
        psi = np.zeros_like(t)
        dpsi = np.zeros_like(t)
        
        psi[mask] = (1 - z[mask]**2)**4
        dpsi[mask] = -8 * z[mask] / width * (1 - z[mask]**2)**3
        
        return psi, dpsi
    
    def _weak_form_transform(
        self,
        Phi: np.ndarray,
        y: np.ndarray,
        t: np.ndarray
    ) -> Tuple[np.ndarray, np.ndarray]:
        """
        Apply weak form transformation.
        
        Q[m,k] = integral(psi_m * Phi_k) dt
        b[m] = -integral(dpsi_m * y) dt  (integration by parts)
        
        Parameters
        ----------
        Phi : np.ndarray
            Feature library (n_samples, n_features)
        y : np.ndarray
            Target vector (n_samples,)
        t : np.ndarray
            Time vector (n_samples,)
        
        Returns
        -------
        Tuple[np.ndarray, np.ndarray]
            (Q, b) - Weak form matrices
        """
        n_samples, n_features = Phi.shape
        
        # Generate test functions
        psi, dpsi = self._generate_test_functions(t)
        
        # Compute weak form matrices via numerical integration
        Q = np.zeros((self.n_test_functions, n_features))
        b = np.zeros(self.n_test_functions)
        
        for m in range(self.n_test_functions):
            # Q[m, k] = integral(psi_m * Phi_k)
            for k in range(n_features):
                Q[m, k] = np.trapz(psi[m] * Phi[:, k], t)
            
            # b[m] = -integral(dpsi_m * y) (integration by parts)
            b[m] = -np.trapz(dpsi[m] * y, t)
        
        return Q, b
    
    def _stlsq_iteration(
        self,
        Q: np.ndarray,
        b: np.ndarray
    ) -> np.ndarray:
        """
        Sequentially Thresholded Least Squares (STLSQ) with relative thresholding.
        
        Algorithm (v4.1 Enhanced):
            1. Initialize with Ridge OLS solution
            2. Compute effective threshold (relative or absolute)
            3. Threshold small coefficients to zero
            4. Refit OLS on remaining support
            5. Repeat until convergence
        
        Parameters
        ----------
        Q : np.ndarray
            Design matrix (n_equations, n_features)
        b : np.ndarray
            Target vector (n_equations,)
        
        Returns
        -------
        np.ndarray
            Sparse coefficient vector
        """
        n_features = Q.shape[1]
        self._convergence_history = []
        
        # Step 1: Initialize with regularized OLS (for stability)
        try:
            ridge = Ridge(alpha=self.ridge_alpha, fit_intercept=False)
            ridge.fit(Q, b)
            xi = ridge.coef_
        except Exception:
            xi = np.linalg.lstsq(Q, b, rcond=None)[0]
        
        self._convergence_history.append(xi.copy())
        
        # Step 2-5: Iterative thresholding with relative threshold
        for iteration in range(self.max_iter):
            self._n_iterations = iteration + 1
            
            # v4.1 FIX: Compute effective threshold
            if self.relative_threshold:
                max_coef = np.max(np.abs(xi))
                if max_coef > 1e-10:
                    effective_threshold = self.threshold * max_coef
                else:
                    effective_threshold = self.threshold
            else:
                effective_threshold = self.threshold
            
            self._effective_threshold = effective_threshold
            
            # Threshold small coefficients
            small_mask = np.abs(xi) < effective_threshold
            xi[small_mask] = 0
            
            # Get active indices
            active = ~small_mask
            
            # Check if any coefficients remain
            if not np.any(active):
                break
            
            # Refit on active support
            Q_active = Q[:, active]
            try:
                xi_active = np.linalg.lstsq(Q_active, b, rcond=None)[0]
            except Exception:
                break
            
            # Update full coefficient vector
            xi_new = np.zeros(n_features)
            xi_new[active] = xi_active
            
            self._convergence_history.append(xi_new.copy())
            
            # Check convergence
            if np.allclose(xi, xi_new, rtol=1e-6):
                xi = xi_new
                break
            
            xi = xi_new
        
        return xi
    
    def get_equation(self) -> str:
        """
        Get string representation of discovered equation.
        
        Returns
        -------
        str
            Equation string with source tags
        """
        if not self._fit_complete:
            return ""
        
        terms = []
        for i, (coef, active) in enumerate(zip(self._coefficients, self._support)):
            if active:
                name = self._library_names[i]
                if abs(coef) > 0.001:
                    terms.append(f"{coef:.3f} * {name}")
        
        if len(terms) == 0:
            return "0"
        
        return " + ".join(terms)
    
    def get_active_terms(self) -> List[Tuple[str, float]]:
        """
        Get list of active terms with coefficients.
        
        Returns
        -------
        List[Tuple[str, float]]
            List of (term_name, coefficient) pairs
        """
        if not self._fit_complete:
            return []
        
        active_terms = []
        for i, (coef, active) in enumerate(zip(self._coefficients, self._support)):
            if active:
                active_terms.append((self._library_names[i], coef))
        
        return active_terms
    
    def predict(self, Phi_new: np.ndarray) -> np.ndarray:
        """
        Make predictions using discovered equation.
        
        Parameters
        ----------
        Phi_new : np.ndarray
            New feature library matrix
        
        Returns
        -------
        np.ndarray
            Predictions
        """
        if not self._fit_complete:
            raise RuntimeError("Must call fit() before predict()")
        
        return Phi_new @ self._coefficients
    
    def print_stlsq_report(self) -> None:
        """
        Print detailed STLSQ results report in v4.2 format.
        """
        if not self._fit_complete:
            print("Fit not yet performed. Run fit() first.")
            return
        
        print("=" * 70)
        print("=== E-WSINDy Results (v4.2 Hybrid) ===")
        print("=" * 70)
        print()
        print(f"Selected terms: {np.sum(self._support)}")
        if self._terms_limited:
            print(f"  [v4.2] Terms were limited (original selection was larger)")
        print()
        
        # Print active terms with source tags
        for name, coef in self.get_active_terms():
            print(f"  {coef:8.3f} * {name}")
        print()
        
        # Print selection analysis
        print("Selection Analysis:")
        print(f"  from_powerlaw: {self._selection_analysis.get('from_powerlaw', 0)}")
        print(f"  from_pysr: {self._selection_analysis['from_pysr']}")
        print(f"  from_variant: {self._selection_analysis['from_variant']}")
        print(f"  from_poly: {self._selection_analysis['from_poly']}")
        print(f"  from_op: {self._selection_analysis['from_op']}")
        print()
        
        print(f"R-squared: {self._r2_score:.4f}")
        print(f"MSE: {self._mse:.6f}")
        print(f"Iterations: {self._n_iterations}")
        print(f"Threshold: {self.threshold}")
        if self._effective_threshold is not None:
            print(f"Effective Threshold: {self._effective_threshold:.6f}")
        print()
        print("=" * 70)

print("EWSINDySTLSQ class v4.2 (Hybrid term limiting) defined.")

---
## Section 2b: E-SINDy Ensemble Class (v4.1 NEW)

**Layer 2 Enhancement:** Bootstrap ensemble for robust sparse selection.

Reference: Fasel et al. (2022) Ensemble-SINDy: Robust sparse model discovery
in the low-data, high-noise limit. Proc. R. Soc. A.

In [ ]:
# ==============================================================================
# E-SINDY ENSEMBLE CLASS (v4.1 NEW - Layer 2)
# ==============================================================================

class ESINDyEnsemble:
    """
    E-SINDy: Ensemble Sparse Identification of Nonlinear Dynamics.
    
    Provides robust sparse selection via bootstrap aggregation with
    inclusion probability estimation. More robust than single-run STLSQ
    for threshold selection and provides natural uncertainty quantification.
    
    Key Features (v4.1):
    - Bootstrap sampling for robust term selection
    - Inclusion probability estimation (natural UQ)
    - Library bagging option for ill-conditioned libraries
    - Coefficient distribution estimation
    
    Reference
    ---------
    Fasel et al. (2022). Ensemble-SINDy: Robust sparse model discovery
    in the low-data, high-noise limit. Proc. R. Soc. A.
    
    Examples
    --------
    >>> ensemble = ESINDyEnsemble(n_models=100, inclusion_threshold=0.9)
    >>> result = ensemble.fit(Phi, y, library_names=names)
    >>> print(f\"Inclusion probs: {result['inclusion_probabilities']}\")
    """
    
    def __init__(
        self,
        n_models: int = 100,
        inclusion_threshold: float = 0.9,
        threshold_range: Tuple[float, float] = (0.05, 0.3),
        library_bagging: bool = False,
        bagging_fraction: float = 0.8,
        use_weak_form: bool = False,
        normalize_columns: bool = True,
        random_state: int = None
    ):
        """
        Initialize E-SINDy Ensemble.
        
        Parameters
        ----------
        n_models : int
            Number of bootstrap models to fit. Default: 100
        inclusion_threshold : float
            Minimum inclusion probability for final selection.
            Terms with P(inclusion) >= threshold are selected. Default: 0.9
        threshold_range : Tuple[float, float]
            Range for random STLSQ threshold sampling. Default: (0.05, 0.3)
        library_bagging : bool
            If True, also subsample library columns. Default: False
        bagging_fraction : float
            Fraction of library to use if library_bagging=True. Default: 0.8
        use_weak_form : bool
            Whether to use weak form in base STLSQ. Default: False
        normalize_columns : bool
            Whether to normalize library columns. Default: True
        random_state : int
            Random seed for reproducibility. Default: None
        """
        self.n_models = n_models
        self.inclusion_threshold = inclusion_threshold
        self.threshold_range = threshold_range
        self.library_bagging = library_bagging
        self.bagging_fraction = bagging_fraction
        self.use_weak_form = use_weak_form
        self.normalize_columns = normalize_columns
        self.random_state = random_state
        
        # Results storage
        self._inclusion_counts = None
        self._inclusion_probabilities = None
        self._coef_samples = None
        self._selected_support = None
        self._final_coefficients = None
        self._library_names = None
        self._r2_score = None
        self._fit_complete = False
    
    def fit(
        self,
        feature_library: np.ndarray,
        y: np.ndarray,
        library_names: List[str] = None,
        t: np.ndarray = None
    ) -> Dict[str, Any]:
        """
        Fit E-SINDy ensemble model.
        
        Parameters
        ----------
        feature_library : np.ndarray
            Feature library matrix (n_samples, n_features)
        y : np.ndarray
            Target vector (n_samples,)
        library_names : List[str], optional
            Feature names with source tags
        t : np.ndarray, optional
            Time vector for weak form
            
        Returns
        -------
        Dict
            - coefficients: Final coefficient vector
            - support: Boolean mask of selected terms
            - inclusion_probabilities: P(term selected) for each term
            - coef_mean: Mean coefficients across ensemble
            - coef_std: Std of coefficients across ensemble
            - selection_analysis: Source attribution
            - r_squared: Final model R-squared
        """
        if self.random_state is not None:
            np.random.seed(self.random_state)
        
        n_samples, n_features = feature_library.shape
        
        # Set library names
        if library_names is None:
            self._library_names = [f'f{i}' for i in range(n_features)]
        else:
            self._library_names = list(library_names)
        
        # Initialize counters
        self._inclusion_counts = np.zeros(n_features)
        self._coef_samples = []
        
        # Bootstrap ensemble
        for model_idx in range(self.n_models):
            # Bootstrap sample rows
            row_idx = np.random.choice(n_samples, n_samples, replace=True)
            Phi_boot = feature_library[row_idx]
            y_boot = y[row_idx]
            names_boot = self._library_names
            
            # Optional library bagging (column subsampling)
            col_idx = np.arange(n_features)
            if self.library_bagging:
                n_cols = int(n_features * self.bagging_fraction)
                col_idx = np.random.choice(n_features, n_cols, replace=False)
                col_idx = np.sort(col_idx)
                Phi_boot = Phi_boot[:, col_idx]
                names_boot = [self._library_names[i] for i in col_idx]
            
            # Random threshold from range
            threshold = np.random.uniform(
                self.threshold_range[0],
                self.threshold_range[1]
            )
            
            # Fit single STLSQ model
            stlsq = EWSINDySTLSQ(
                threshold=threshold,
                use_weak_form=self.use_weak_form,
                normalize_columns=self.normalize_columns,
                relative_threshold=True
            )
            
            try:
                result = stlsq.fit(Phi_boot, y_boot, library_names=names_boot, t=t)
                
                # Map back to full feature space if library bagging
                full_support = np.zeros(n_features, dtype=bool)
                full_coef = np.zeros(n_features)
                
                if self.library_bagging:
                    full_support[col_idx] = result['support']
                    full_coef[col_idx] = result['coefficients']
                else:
                    full_support = result['support']
                    full_coef = result['coefficients']
                
                # Record inclusion
                self._inclusion_counts += full_support.astype(float)
                self._coef_samples.append(full_coef)
                
            except Exception as e:
                # Skip failed fits
                continue
        
        # Compute inclusion probabilities
        n_successful = len(self._coef_samples)
        if n_successful == 0:
            raise ValueError("All ensemble models failed to fit")
        
        self._inclusion_probabilities = self._inclusion_counts / n_successful
        
        # Select terms with high inclusion probability
        self._selected_support = self._inclusion_probabilities >= self.inclusion_threshold
        
        # Compute coefficient statistics
        coef_array = np.array(self._coef_samples)
        coef_mean = np.mean(coef_array, axis=0)
        coef_std = np.std(coef_array, axis=0)
        
        # Final refit on selected support (unbiased)
        if np.any(self._selected_support):
            Phi_selected = feature_library[:, self._selected_support]
            try:
                final_coef_active = np.linalg.lstsq(
                    Phi_selected, y, rcond=None
                )[0]
                self._final_coefficients = np.zeros(n_features)
                self._final_coefficients[self._selected_support] = final_coef_active
            except Exception:
                # Use mean coefficients as fallback
                self._final_coefficients = coef_mean * self._selected_support
        else:
            self._final_coefficients = np.zeros(n_features)
        
        # Compute R-squared
        y_pred = feature_library @ self._final_coefficients
        ss_tot = np.sum((y - np.mean(y))**2)
        ss_res = np.sum((y - y_pred)**2)
        self._r2_score = 1 - ss_res / ss_tot if ss_tot > 0 else 0.0
        
        # Selection analysis
        selection_analysis = self._analyze_sources(
            self._selected_support, self._library_names
        )
        
        self._fit_complete = True
        
        return {
            'coefficients': self._final_coefficients,
            'support': self._selected_support,
            'inclusion_probabilities': self._inclusion_probabilities,
            'coef_mean': coef_mean,
            'coef_std': coef_std,
            'n_active_terms': int(np.sum(self._selected_support)),
            'selection_analysis': selection_analysis,
            'r_squared': self._r2_score,
            'r2_score': self._r2_score,
            'n_successful_models': n_successful,
            'equation': self.get_equation()
        }
    
    def _analyze_sources(
        self,
        support: np.ndarray,
        library_names: List[str]
    ) -> Dict[str, int]:
        """Analyze selection sources (same as EWSINDySTLSQ)."""
        sources = {
            'from_powerlaw': 0,
            'from_pysr': 0,
            'from_variant': 0,
            'from_poly': 0,
            'from_op': 0,
            'from_unknown': 0,
            'total_selected': 0
        }
        
        selected_indices = np.where(support)[0]
        sources['total_selected'] = len(selected_indices)
        
        for idx in selected_indices:
            if idx >= len(library_names):
                sources['from_unknown'] += 1
                continue
            
            name = library_names[idx]
            if name.startswith('[PowLaw]'):
                sources['from_powerlaw'] += 1
            elif name.startswith('[PySR]'):
                sources['from_pysr'] += 1
            elif name.startswith('[Var]'):
                sources['from_variant'] += 1
            elif name.startswith('[Poly]'):
                sources['from_poly'] += 1
            elif name.startswith('[Op]'):
                sources['from_op'] += 1
            else:
                sources['from_unknown'] += 1
        
        return sources
    
    def get_equation(self) -> str:
        """Get string representation of discovered equation."""
        if not self._fit_complete:
            return ""
        
        terms = []
        for i, (coef, active) in enumerate(zip(self._final_coefficients, self._selected_support)):
            if active and abs(coef) > 1e-10:
                name = self._library_names[i]
                prob = self._inclusion_probabilities[i]
                terms.append(f"{coef:.4f} * {name} (P={prob:.2f})")
        
        return " + ".join(terms) if terms else "0"
    
    def get_high_confidence_terms(
        self,
        min_probability: float = 0.9
    ) -> List[Tuple[str, float, float]]:
        """
        Get terms with high inclusion probability.
        
        Returns
        -------
        List[Tuple[str, float, float]]
            List of (name, coefficient, inclusion_probability)
        """
        if not self._fit_complete:
            return []
        
        results = []
        for i, prob in enumerate(self._inclusion_probabilities):
            if prob >= min_probability:
                results.append((
                    self._library_names[i],
                    self._final_coefficients[i],
                    prob
                ))
        
        return sorted(results, key=lambda x: -x[2])
    
    def print_ensemble_report(self) -> None:
        """Print detailed ensemble results."""
        if not self._fit_complete:
            print("Model not yet fitted.")
            return
        
        print("=" * 70)
        print(" E-SINDy Ensemble Report")
        print("=" * 70)
        print()
        print(f"Models fitted: {len(self._coef_samples)}")
        print(f"Inclusion threshold: {self.inclusion_threshold}")
        print(f"Selected terms: {np.sum(self._selected_support)}")
        print(f"R-squared: {self._r2_score:.4f}")
        print()
        print("High-confidence terms (P >= 0.9):")
        for name, coef, prob in self.get_high_confidence_terms(0.9):
            print(f"  {coef:10.4f} * {name:30s} (P={prob:.3f})")
        print()
        print("=" * 70)

print("ESINDyEnsemble class v4.1 defined.")

---
## Section 2c: EBIC Threshold Selection (v4.1 NEW)

**Layer 3 Enhancement:** Automatic threshold selection via Extended BIC.

The Extended Bayesian Information Criterion (EBIC) balances model fit
against complexity, with a penalty term that accounts for the large
search space in sparse regression.

In [ ]:
# ==============================================================================
# EBIC THRESHOLD SELECTION (v4.1 NEW - Layer 3)
# ==============================================================================

def select_threshold_by_ebic(
    feature_library: np.ndarray,
    y: np.ndarray,
    library_names: List[str] = None,
    thresholds: List[float] = None,
    gamma: float = 0.5,
    use_weak_form: bool = False,
    normalize_columns: bool = True,
    verbose: bool = False
) -> Dict[str, Any]:
    """
    Select optimal STLSQ threshold using Extended BIC.
    
    The Extended BIC (Chen & Chen, 2008) adds a penalty term that scales
    with the size of the model space, making it suitable for high-dimensional
    sparse regression where the number of potential models is large.
    
    EBIC = n * log(MSE) + k * log(n) + 2 * gamma * k * log(p)
    
    where:
    - n = number of samples
    - k = number of selected terms
    - p = total number of candidate terms
    - gamma in [0, 1] controls penalty strength (0.5 recommended)
    
    Parameters
    ----------
    feature_library : np.ndarray
        Feature library matrix (n_samples, n_features)
    y : np.ndarray
        Target vector (n_samples,)
    library_names : List[str], optional
        Feature names
    thresholds : List[float], optional
        Candidate thresholds to evaluate.
        Default: [0.01, 0.02, 0.05, 0.1, 0.2, 0.3, 0.5]
    gamma : float
        EBIC penalty parameter. Higher values favor sparser models.
        Default: 0.5
    use_weak_form : bool
        Whether to use weak form. Default: False
    normalize_columns : bool
        Whether to normalize columns. Default: True
    verbose : bool
        Print details for each threshold. Default: False
        
    Returns
    -------
    Dict
        - best_threshold: Optimal threshold
        - best_ebic: EBIC at optimal threshold
        - best_result: Full result dict from best model
        - all_results: Results for all thresholds
    
    Reference
    ---------
    Chen, J., & Chen, Z. (2008). Extended Bayesian information criteria
    for model selection with large model spaces. Biometrika, 95(3), 759-771.
    """
    if thresholds is None:
        thresholds = [0.01, 0.02, 0.05, 0.1, 0.2, 0.3, 0.5]
    
    n_samples, n_features = feature_library.shape
    
    if verbose:
        print("EBIC Threshold Selection")
        print("=" * 60)
        print(f"{'Threshold':<12} {'k (terms)':<12} {'MSE':<15} {'EBIC':<15}")
        print("-" * 60)
    
    best_ebic = np.inf
    best_threshold = thresholds[0]
    best_result = None
    all_results = []
    
    for threshold in thresholds:
        # Fit STLSQ with this threshold
        stlsq = EWSINDySTLSQ(
            threshold=threshold,
            use_weak_form=use_weak_form,
            normalize_columns=normalize_columns,
            relative_threshold=True
        )
        
        try:
            result = stlsq.fit(feature_library, y, library_names=library_names)
        except Exception:
            continue
        
        k = result['n_active_terms']
        mse = result['mse']
        
        # Handle edge cases
        if k == 0:
            # No terms selected - use total variance as MSE
            mse = np.var(y)
            ebic = np.inf
        elif mse <= 0:
            # Perfect fit
            ebic = -np.inf
        else:
            # Compute EBIC
            # EBIC = n * log(MSE) + k * log(n) + 2 * gamma * k * log(p)
            ebic = (
                n_samples * np.log(mse) +
                k * np.log(n_samples) +
                2 * gamma * k * np.log(n_features)
            )
        
        result['ebic'] = ebic
        result['threshold_used'] = threshold
        all_results.append(result)
        
        if verbose:
            print(f"{threshold:<12.3f} {k:<12d} {mse:<15.6f} {ebic:<15.2f}")
        
        if ebic < best_ebic:
            best_ebic = ebic
            best_threshold = threshold
            best_result = result
    
    if verbose:
        print("-" * 60)
        print(f"Best threshold: {best_threshold} (EBIC={best_ebic:.2f})")
        print("=" * 60)
    
    return {
        'best_threshold': best_threshold,
        'best_ebic': best_ebic,
        'best_result': best_result,
        'all_results': all_results
    }


def select_threshold_by_cv(
    feature_library: np.ndarray,
    y: np.ndarray,
    library_names: List[str] = None,
    thresholds: List[float] = None,
    n_folds: int = 5,
    use_weak_form: bool = False,
    normalize_columns: bool = True,
    verbose: bool = False
) -> Dict[str, Any]:
    """
    Select optimal STLSQ threshold using cross-validation.
    
    Parameters
    ----------
    feature_library : np.ndarray
        Feature library matrix
    y : np.ndarray
        Target vector
    library_names : List[str], optional
        Feature names
    thresholds : List[float], optional
        Candidate thresholds. Default: [0.01, 0.02, 0.05, 0.1, 0.2, 0.3, 0.5]
    n_folds : int
        Number of CV folds. Default: 5
    use_weak_form : bool
        Whether to use weak form. Default: False
    normalize_columns : bool
        Whether to normalize columns. Default: True
    verbose : bool
        Print details. Default: False
        
    Returns
    -------
    Dict
        - best_threshold: Optimal threshold
        - best_cv_score: Mean CV R-squared at optimal threshold
        - cv_results: Results for all thresholds
    """
    from sklearn.model_selection import KFold
    
    if thresholds is None:
        thresholds = [0.01, 0.02, 0.05, 0.1, 0.2, 0.3, 0.5]
    
    n_samples = feature_library.shape[0]
    kf = KFold(n_splits=n_folds, shuffle=True, random_state=42)
    
    if verbose:
        print("Cross-Validation Threshold Selection")
        print("=" * 60)
        print(f"{'Threshold':<12} {'Mean R2':<12} {'Std R2':<12} {'Mean k':<12}")
        print("-" * 60)
    
    best_cv_score = -np.inf
    best_threshold = thresholds[0]
    cv_results = []
    
    for threshold in thresholds:
        fold_r2s = []
        fold_ks = []
        
        for train_idx, val_idx in kf.split(feature_library):
            X_train = feature_library[train_idx]
            y_train = y[train_idx]
            X_val = feature_library[val_idx]
            y_val = y[val_idx]
            
            stlsq = EWSINDySTLSQ(
                threshold=threshold,
                use_weak_form=use_weak_form,
                normalize_columns=normalize_columns,
                relative_threshold=True
            )
            
            try:
                result = stlsq.fit(X_train, y_train, library_names=library_names)
                
                # Evaluate on validation set
                y_pred = X_val @ result['coefficients']
                ss_tot = np.sum((y_val - np.mean(y_val))**2)
                ss_res = np.sum((y_val - y_pred)**2)
                val_r2 = 1 - ss_res / ss_tot if ss_tot > 0 else 0.0
                
                fold_r2s.append(val_r2)
                fold_ks.append(result['n_active_terms'])
            except Exception:
                continue
        
        if len(fold_r2s) > 0:
            mean_r2 = np.mean(fold_r2s)
            std_r2 = np.std(fold_r2s)
            mean_k = np.mean(fold_ks)
            
            cv_results.append({
                'threshold': threshold,
                'mean_r2': mean_r2,
                'std_r2': std_r2,
                'mean_k': mean_k
            })
            
            if verbose:
                print(f"{threshold:<12.3f} {mean_r2:<12.4f} {std_r2:<12.4f} {mean_k:<12.1f}")
            
            if mean_r2 > best_cv_score:
                best_cv_score = mean_r2
                best_threshold = threshold
    
    if verbose:
        print("-" * 60)
        print(f"Best threshold: {best_threshold} (CV R2={best_cv_score:.4f})")
        print("=" * 60)
    
    return {
        'best_threshold': best_threshold,
        'best_cv_score': best_cv_score,
        'cv_results': cv_results
    }

print("EBIC and CV threshold selection functions v4.1 defined.")

---
## Section 3: Internal Tests

In [ ]:
# ==============================================================================
# TEST CONTROL FLAG
# ==============================================================================

_RUN_TESTS = False  # Set to True to run internal tests

if _RUN_TESTS:
    print("=" * 70)
    print(" RUNNING INTERNAL TESTS FOR 07_EWSINDy_STLSQ v4.1")
    print("=" * 70)

In [ ]:
# ==============================================================================
# TEST 1: Basic STLSQ with Standard Library
# ==============================================================================

if _RUN_TESTS:
    print()
    print_section_header("Test 1: Basic STLSQ with Standard Library")
    
    np.random.seed(42)
    n_samples = 200
    
    x = np.random.uniform(0.1, 2, n_samples)
    y = 3*x + 2*x**2 + 0.01*np.random.randn(n_samples)
    
    # Standard library (no source tags)
    Phi = np.column_stack([np.ones(n_samples), x, x**2, x**3, x**4])
    library_names = ['1', 'x', 'x^2', 'x^3', 'x^4']
    
    model = EWSINDySTLSQ(threshold=0.1, use_weak_form=False)
    result = model.fit(Phi, y, library_names=library_names)
    
    print(f"True: y = 3*x + 2*x^2")
    print(f"Discovered: {result['equation']}")
    print(f"R-squared: {result['r_squared']:.4f}")
    print(f"Active terms: {result['n_active_terms']}")
    print()
    
    if result['r_squared'] > 0.99:
        print("[PASS] High accuracy achieved")
    else:
        print("[WARNING] Accuracy lower than expected")

In [ ]:
# ==============================================================================
# TEST 2: Source Attribution with Augmented Library
# ==============================================================================

if _RUN_TESTS:
    print()
    print_section_header("Test 2: Source Attribution with Augmented Library")
    
    np.random.seed(42)
    n_samples = 200
    
    x = np.random.uniform(0.1, 2, n_samples)
    z = np.random.uniform(0.1, 2, n_samples)
    
    # True equation: y = 0.5*x^2 + sin(z)
    y = 0.5*x**2 + np.sin(z) + 0.01*np.random.randn(n_samples)
    
    # Simulated augmented library with source tags
    Phi = np.column_stack([
        x**2,           # [PySR] x**2
        np.sin(z),      # [PySR] sin(z)
        np.sin(x),      # [Var] sin(x)
        np.ones(n_samples),  # [Poly] 1
        x,              # [Poly] x
        z,              # [Poly] z
        x*z,            # [Poly] x*z
        np.cos(x),      # [Op] cos(x)
        np.cos(z)       # [Op] cos(z)
    ])
    
    library_names = [
        '[PySR] x**2',
        '[PySR] sin(z)',
        '[Var] sin(x)',
        '[Poly] 1',
        '[Poly] x',
        '[Poly] z',
        '[Poly] x*z',
        '[Op] cos(x)',
        '[Op] cos(z)'
    ]
    
    model = EWSINDySTLSQ(threshold=0.1, use_weak_form=False)
    result = model.fit(Phi, y, library_names=library_names)
    
    print(f"True: y = 0.5*x^2 + sin(z)")
    print()
    print("Selected terms:")
    for name, coef in model.get_active_terms():
        print(f"  {coef:8.3f} * {name}")
    print()
    print(f"Selection Analysis: {result['selection_analysis']}")
    print(f"R-squared: {result['r_squared']:.4f}")
    print()
    
    # Check that PySR terms were selected
    analysis = result['selection_analysis']
    if analysis['from_pysr'] >= 2:
        print("[PASS] PySR terms correctly selected")
    else:
        print(f"[INFO] Selected {analysis['from_pysr']} PySR terms")

In [ ]:
# ==============================================================================
# TEST 3: Noise Robustness with Weak Form
# ==============================================================================

if _RUN_TESTS:
    print()
    print_section_header("Test 3: Noise Robustness with Weak Form")
    
    np.random.seed(42)
    n_samples = 300
    
    t = np.linspace(0, 1, n_samples)
    x = np.sin(2 * np.pi * t)
    
    # True dynamics: dy/dt = x (approximately)
    y_clean = x.copy()
    
    noise_levels = [0.01, 0.05, 0.10]
    
    Phi = np.column_stack([np.ones(n_samples), x, x**2])
    library_names = ['[Poly] 1', '[Poly] x', '[Poly] x^2']
    
    print(f"{'Noise':<12} {'R2 (strong)':<15} {'R2 (weak)':<15}")
    print("-" * 45)
    
    for noise_level in noise_levels:
        y_noisy = y_clean + noise_level * np.random.randn(n_samples)
        
        # Strong form
        model_strong = EWSINDySTLSQ(threshold=0.1, use_weak_form=False)
        result_strong = model_strong.fit(Phi, y_noisy, library_names=library_names)
        
        # Weak form
        model_weak = EWSINDySTLSQ(threshold=0.1, use_weak_form=True)
        result_weak = model_weak.fit(Phi, y_noisy, t=t, library_names=library_names)
        
        print(f"{noise_level:<12.2f} {result_strong['r_squared']:<15.4f} {result_weak['r_squared']:<15.4f}")
    
    print()
    print("[INFO] Weak form should be more robust at higher noise levels")

In [ ]:
# ==============================================================================
# TEST 4: analyze_selection_sources Method
# ==============================================================================

if _RUN_TESTS:
    print()
    print_section_header("Test 4: analyze_selection_sources Method")
    
    # Create mock support and library names
    support = np.array([True, True, False, True, False, False, True, False])
    library_names = [
        '[PySR] sin(x)',
        '[PySR] x**2',
        '[Var] sin(y)',
        '[Poly] 1',
        '[Poly] x',
        '[Poly] y',
        '[Op] exp(x)',
        '[Op] cos(x)'
    ]
    
    model = EWSINDySTLSQ()
    analysis = model.analyze_selection_sources(support, library_names)
    
    print("Support mask:")
    for i, (s, n) in enumerate(zip(support, library_names)):
        status = "SELECTED" if s else "        "
        print(f"  {i}: {status} {n}")
    print()
    print(f"Analysis result: {analysis}")
    print()
    
    # Expected: 2 PySR, 0 Var, 1 Poly, 1 Op
    expected = {'from_pysr': 2, 'from_variant': 0, 'from_poly': 1, 'from_op': 1}
    
    all_correct = True
    for key, expected_val in expected.items():
        actual_val = analysis[key]
        status = "PASS" if actual_val == expected_val else "FAIL"
        if status == "FAIL":
            all_correct = False
        print(f"  {key}: expected={expected_val}, actual={actual_val} [{status}]")
    
    print()
    if all_correct:
        print("[PASS] All source attribution correct")
    else:
        print("[FAIL] Some source attributions incorrect")

In [ ]:
# ==============================================================================
# TEST 5: Full Report Output
# ==============================================================================

if _RUN_TESTS:
    print()
    print_section_header("Test 5: Full Report Output")
    
    np.random.seed(42)
    n_samples = 200
    
    x = np.random.uniform(0.1, 2, n_samples)
    z = np.random.uniform(0.1, 2, n_samples)
    y = 0.5*x**2 + np.sin(z) + 0.01*np.random.randn(n_samples)
    
    # Augmented library
    Phi = np.column_stack([
        x**2,
        np.sin(z),
        np.ones(n_samples),
        x,
        z
    ])
    
    library_names = [
        '[PySR] x**2',
        '[PySR] sin(z)',
        '[Poly] 1',
        '[Poly] x',
        '[Poly] z'
    ]
    
    model = EWSINDySTLSQ(threshold=0.1, use_weak_form=False)
    result = model.fit(Phi, y, library_names=library_names)
    
    # Print full report
    model.print_stlsq_report()

---
## Section 4: Module Summary

In [ ]:
# ==============================================================================
# MODULE SUMMARY
# ==============================================================================

print("=" * 70)
print(" 07_EWSINDy_STLSQ.ipynb v4.1 - Module Summary")
print("=" * 70)
print()
print("THREE-LAYER SPARSE SELECTION ENHANCEMENT")
print("-" * 70)
print()
print("Layer 1: EWSINDySTLSQ (Relative Threshold)")
print("  - Scale-invariant STLSQ: threshold = base * max(|coef|)")
print("  - Robust to coefficient magnitude variations")
print("  - Parameters: threshold, relative_threshold, ridge_alpha")
print()
print("Layer 2: ESINDyEnsemble (Bootstrap Selection)")
print("  - Bootstrap aggregation for robust term selection")
print("  - Inclusion probability estimation (natural UQ)")
print("  - Optional library bagging for ill-conditioned libraries")
print("  - Parameters: n_models, inclusion_threshold, threshold_range")
print()
print("Layer 3: EBIC/CV Threshold Selection")
print("  - select_threshold_by_ebic(): Extended BIC criterion")
print("  - select_threshold_by_cv(): Cross-validation criterion")
print("  - Automatic threshold tuning")
print()
print("=" * 70)
print("CLASS: EWSINDySTLSQ")
print("=" * 70)
print()
print("Key Parameters:")
print("  threshold: Base STLSQ sparsity threshold (default: 0.1)")
print("  relative_threshold: Use relative thresholding (default: True)")
print("  use_weak_form: Enable weak form (default: True)")
print("  normalize_columns: Normalize library columns (default: True)")
print("  ridge_alpha: Ridge regularization strength (default: 1e-6)")
print()
print("Main Methods:")
print("  fit(feature_library, y, library_names=None, t=None) -> Dict")
print("  analyze_selection_sources(support, library_names) -> Dict")
print("  get_equation() -> str")
print("  print_stlsq_report()")
print()
print("=" * 70)
print("CLASS: ESINDyEnsemble")
print("=" * 70)
print()
print("Key Parameters:")
print("  n_models: Number of bootstrap models (default: 100)")
print("  inclusion_threshold: Min probability for selection (default: 0.9)")
print("  threshold_range: STLSQ threshold sampling range (default: (0.05, 0.3))")
print("  library_bagging: Subsample columns too (default: False)")
print()
print("Main Methods:")
print("  fit(feature_library, y, library_names=None) -> Dict")
print("  get_high_confidence_terms(min_prob=0.9) -> List[Tuple]")
print("  print_ensemble_report()")
print()
print("=" * 70)
print("FUNCTIONS: Threshold Selection")
print("=" * 70)
print()
print("select_threshold_by_ebic(Phi, y, thresholds=None, gamma=0.5) -> Dict")
print("  EBIC = n*log(MSE) + k*log(n) + 2*gamma*k*log(p)")
print()
print("select_threshold_by_cv(Phi, y, thresholds=None, n_folds=5) -> Dict")
print("  Cross-validation for threshold selection")
print()
print("=" * 70)
print("USAGE EXAMPLES")
print("=" * 70)
print("""
# Layer 1: Basic STLSQ with relative threshold
stlsq = EWSINDySTLSQ(threshold=0.1, relative_threshold=True)
result = stlsq.fit(Phi, y, library_names=names)

# Layer 2: Ensemble selection for robust UQ
ensemble = ESINDyEnsemble(n_models=100, inclusion_threshold=0.9)
result = ensemble.fit(Phi, y, library_names=names)
print(f"High-confidence terms: {ensemble.get_high_confidence_terms()}")

# Layer 3: Automatic threshold selection
ebic_result = select_threshold_by_ebic(Phi, y, verbose=True)
best_threshold = ebic_result['best_threshold']
""")
print()
print("=" * 70)
print("References:")
print("  - Messenger & Bortz (2021). Weak SINDy. JCP 443.")
print("  - Fasel et al. (2022). Ensemble-SINDy. Proc. R. Soc. A.")
print("  - Chen & Chen (2008). Extended BIC. Biometrika 95(3).")
print("=" * 70)
print()
print("Module loaded successfully. Import via: %run 07_EWSINDy_STLSQ.ipynb")
print("=" * 70)